# 通道套利策略：Bollinger Band

**Bollinger Band(布林帶)** 的原理，是假設股票的價格是服從**常態分佈**

- 從分配的中心點，也就是平均值，往**兩邊加減兩倍的標準差的範圍，占了所有範圍大約 95% 的比例**
- 短期的上漲或下跌，勢必會回復到常態
- 給予一段觀察期(通常是20天)，每天用前20日的資料算出平均值與標準差



![](https://drive.google.com/uc?export=download&id=1YukQViaMakj_j_5C47QzlUYutE7_AAto)

Bollinger Band 的交易策略是：

    1. 若收盤價大於過去 20 日收盤價的最大值，代表**不正常上漲**，所以就做多股票，直到股價跌破中心點平倉
    2. 若收盤價小於過去 20 日收盤價的最小值，代表**不正常下跌**，所以就做空股票，直到股價漲破中心點平倉

***備注：非投資建議，本課程提供的資料及交易策略，只可作為參考用途，學員在投資前，務請運用個人獨立思考做出抉擇，如因此招致任何損失，概與本課程無涉。**



In [ ]:
import pandas as pd

df = pd.read_excel("2884_歷史資料.xlsx")
df

# 先來計算上，下，與中軌


In [ ]:
df["收盤價"].rolling(20).std() * 2

In [ ]:
df["max20d"] = df["收盤價"] + 2 * df["收盤價"].rolling(20).std()
df

In [ ]:
df["min20d"] = df["收盤價"] - 2 * df["收盤價"].rolling(20).std()
df

In [ ]:
df["收盤價"].rolling(20).mean()

In [ ]:
df["mean20d"] = df["收盤價"].rolling(20).mean()
df

In [ ]:
df2 = df.set_index("日期")
df2

In [ ]:
df2["max20d"] = df2["max20d"].shift(1)
df2["min20d"] = df2["min20d"].shift(1)
df2["mean20d"] = df2["mean20d"].shift(1)
df2

In [ ]:
df2[["收盤價", "max20d", "min20d", "mean20d"]]

In [ ]:
df2[["收盤價", "max20d", "min20d", "mean20d"]]["2017-02-09":]

In [ ]:
df3 = df2[["收盤價", "max20d", "min20d", "mean20d"]]["2017-02-09":]
df3

In [ ]:
for index, row in df3.iterrows():
    #print(index)
    print(row["max20d"])

In [ ]:
import numpy as np
df3["交易訊號"] = ""
df3["損益"] = np.nan
df3

In [ ]:
# 目前是否已做多或做空的訊號
hold_flag = False
# 買入股票的訊號
long_flag = False
# 做空股票的訊號
short_flag = False
# 交易股數
shares = 1000
# 記錄總收益
balance = 0


for index, row in df3.iterrows():
    print(index)
    print(row)
    # 若目前沒有進行交易
    if hold_flag == False:
        # 價格低於下限
        if row["收盤價"] <= row["min20d"]:
            # 顯示做多
            df3.loc[index, "交易訊號"] = "Long"
            # 計算總收益
            balance = balance - (row["收盤價"] * shares)
            # 花費現金購買股票
            df3.loc[index, "損益"] = balance
            # 開啓交易訊號
            long_flag = True
            hold_flag = True
            # 價格高於上限
        elif row["收盤價"] >= row["max20d"]:
            # 顯示做空
            df3.loc[index, "交易訊號"] = "Short"
            # 做空，現金增加
            balance = balance + (row["收盤價"] * shares)
            df3.loc[index, "損益"] = balance
            # 開啓交易訊號
            short_flag = True
            hold_flag = True
        else:
            df3.loc[index, "交易訊號"] = "---"
            df3.loc[index, "損益"] = balance
    # 若目前有進行交易
    elif hold_flag == True:
       # 若現在是做多，而且價格大於等於平均值，平倉
       if long_flag == True and row["收盤價"] >= row["mean20d"]:
            df3.loc[index, "交易訊號"] = "Offset"
            # 做多在平倉時，現金增加
            balance = balance + row["收盤價"] * shares
            df3.loc[index, "損益"] = balance
            hold_flag = False
            long_flag = False
            balance = 0
       # 若現在是做空，而且價格小於等於平均值，平倉
       elif short_flag == True and row["收盤價"] <= row["mean20d"]:
            df3.loc[index, "交易訊號"] = "Offset"
            # 做空在平倉時，現金減少
            balance = balance - row["收盤價"] * shares
            df3.loc[index, "損益"] = balance
            hold_flag = False
            short_flag = False
            balance = 0
       else:
            df3.loc[index, "交易訊號"] = "---"
            df3.loc[index, "損益"] = balance
    
    print(balance)

In [ ]:
df3

In [ ]:
df3[df3["交易訊號"] == "Offset"]

In [ ]:
result_df = df3[df3["交易訊號"] == "Offset"]
result_df

In [ ]:
win_rate = df3[(df3["交易訊號"] == "Offset") & (df3["損益"] > 0)].shape[0] / df3[df3["交易訊號"] == "Offset"].shape[0]
win_rate

In [ ]:
profit = df3[(df3["交易訊號"] == "Offset")]["損益"].sum()
profit

In [ ]:
df3["纍積收益"] =  df3[df3["交易訊號"] == "Offset"]["損益"].cumsum()
df4 = df3[df3["交易訊號"] == "Offset"]
df4

In [ ]:
import xlwings as xw

wb = xw.Book()

sheet = wb.sheets.add(name="Bollinger Band 回測結果")
sheet.range("A1").value = df4
sheet.range("K1").value = "獲勝率"
sheet.range("L1").value = win_rate
sheet.range("K2").value = "纍計收益"
sheet.range("L2").value = profit

wb.save("2884 Bollinger Band 回測")